In [1]:
import arviz as az
import xarray as xr

from generate_data import generate_data
from utils import StanModel_cache

In [2]:
n = 70
Years_indiv, Mean_RT_comp_Indiv, Mean_RT_incomp_Indiv = generate_data(8, n)
dims = {"y_obs_comp": ["subject"], "y_obs_incomp": ["subject"]}
log_lik_dict = {"y_obs_comp": "log_lik_comp", "y_obs_incomp": "log_lik_comp"}
data = {
    "n": n,
    "y_obs_comp": Mean_RT_comp_Indiv,
    "y_obs_incomp": Mean_RT_incomp_Indiv,
    "age": Years_indiv,
    "mean_rt_c": Mean_RT_comp_Indiv.mean(),
    "mean_rt_i": Mean_RT_incomp_Indiv.mean(),
}

In [3]:
exp_code = """
data {
    int<lower=0> n;
    real y_obs_comp[n];
    real y_obs_incomp[n];
    int<lower=0> age[n];
    real mean_rt_c;
    real mean_rt_i;
}

parameters {
    real b;
    real<lower=0> sigma;
    real<lower=0> a_c;
    real<lower=0> a_i;
    real g_c;
    real g_i;
}

transformed parameters {
    real mu_c[n];
    real mu_i[n];
    for (j in 1:n) {
        mu_c[j] = a_c*exp(-b*age[j]) + g_c;
        mu_i[j] = a_i*exp(-b*age[j]) + g_i;
    }
}

model {
    a_c ~ cauchy(0, 5);
    a_i ~ cauchy(0, 5);
    b ~ normal(1, 1);
    g_c ~ normal(mean_rt_c, .5);
    g_i ~ normal(mean_rt_i, .5);
    sigma ~ normal(0, .2);
    y_obs_comp ~ normal(mu_c, sigma);
    y_obs_incomp ~ normal(mu_i, sigma);
}

generated quantities {
    real log_lik_comp[n];
    real log_lik_incomp[n];

    for (j in 1:n) {
        log_lik_comp[j] = normal_lpdf(y_obs_comp[j] | mu_c[j], sigma);
        log_lik_incomp[j] = normal_lpdf(y_obs_incomp[j] | mu_i[j], sigma);
    }
}
"""

In [4]:
stan_model = StanModel_cache(model_code=exp_code)
fit = stan_model.sampling(data=data, iter=4000, control={"adapt_delta" : 0.9})

Using cached StanModel


In [5]:
idata_exp = az.from_pystan(fit, dims=dims, log_likelihood=log_lik_dict)

The pointwise log likelihood stored is the following (both models have the same variables and shape, only the exponential model is shown)

In [6]:
idata_exp.log_likelihood

<xarray.Dataset>
Dimensions:       (chain: 4, draw: 2000, subject: 70)
Coordinates:
  * chain         (chain) int64 0 1 2 3
  * draw          (draw) int64 0 1 2 3 4 5 6 ... 1994 1995 1996 1997 1998 1999
  * subject       (subject) int64 0 1 2 3 4 5 6 7 8 ... 62 63 64 65 66 67 68 69
Data variables:
    y_obs_comp    (chain, draw, subject) float64 -0.9269 -1.255 ... -2.182
    y_obs_incomp  (chain, draw, subject) float64 -0.9269 -1.255 ... -2.182
Attributes:
    created_at:                 2020-03-20T11:50:13.447530
    arviz_version:              0.7.0
    inference_library:          pystan
    inference_library_version:  2.19.1.1

IC calculation and model comparison starts here

In [7]:
log_lik_exp = idata_exp.log_likelihood

print("Leave one *observation* out cross validation (whole model)")
condition_dim = xr.DataArray(["compatible", "incompatible"], name="condition")
idata_exp.sample_stats["log_likelihood"] = xr.concat((log_lik_exp.y_obs_comp, log_lik_exp.y_obs_incomp), dim=condition_dim)
print(az.loo(idata_exp))

print("\n\nLeave one *subject* out cross validation (whole model)")
idata_exp.sample_stats["log_likelihood"] = log_lik_exp.to_array().sum("variable")
print(az.loo(idata_exp))

print("\n\nLeave one observation out cross validation (y_obs_comp only)")
idata_exp.sample_stats["log_likelihood"] = log_lik_exp.y_obs_comp
print(az.loo(idata_exp))

print("\n\nLeave one observation out cross validation (y_obs_incomp only)")
idata_exp.sample_stats["log_likelihood"] = log_lik_exp.y_obs_incomp
print(az.loo(idata_exp))

Leave one *observation* out cross validation (whole model)
Computed from 8000 by 140 log-likelihood matrix

         Estimate       SE
elpd_loo  -212.85    11.95
p_loo        5.72        -


Leave one *subject* out cross validation (whole model)


/home/oriol/venvs/arviz-dev/lib/python3.6/site-packages/arviz/stats/stats.py:532: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "


Computed from 8000 by 70 log-likelihood matrix

         Estimate       SE
elpd_loo  -216.40    17.63
p_loo       11.74        -

There has been a warning during the calculation. Please check the results.


Leave one observation out cross validation (y_obs_comp only)
Computed from 8000 by 70 log-likelihood matrix

         Estimate       SE
elpd_loo  -106.42     8.45
p_loo        2.86        -


Leave one observation out cross validation (y_obs_incomp only)
Computed from 8000 by 70 log-likelihood matrix

         Estimate       SE
elpd_loo  -106.42     8.45
p_loo        2.86        -
